In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

from tqdm.notebook import tqdm

### Calibrate parcels
This notebook calibrates the demand generation model by fitting the marginal distributions of the ADM report. It basically implements some kind of uniform seed IPF.

In [ ]:
# Manage inputs and outputs
persons_path = "../../resources/external/population_2022/lead_2022_100pct_persons.csv"
homes_path = "../../resources/external/population_2022/lead_2022_100pct_homes.gpkg"
metropole_path = "../../results/metropole.gpkg"
output_path = "../../results/parcel_model.parquet"

crs = "EPSG:2154"
random_seed = 0

if "snakemake" in locals():
    persons_path = snakemake.input["persons"]
    homes_path = snakemake.input["homes"]
    metropole_path = snakemake.input["metropole"]
    output_path = snakemake.output[0]
    
    params = snakemake.params[0] if len(snakemake.params) == 1 and len(snakemake.params.keys()) == 0 else snakemake.params

    if "crs" in params:
        crs = params["crs"]

    if "random_seed" in params:
        random_seed = params["random_seed"]

In [ ]:
# Filter out persons not within the Grand Lyon Metropole
df_homes = gpd.read_file(homes_path)[["household_id", "geometry"]].to_crs(crs)
df_metropole = gpd.read_file(metropole_path).to_crs(crs)

df_homes = gpd.sjoin(df_homes, df_metropole, predicate = "within")

In [ ]:
# Filter for persons over 18 to find reference persons
df_persons = pd.read_csv(persons_path, sep = ";", usecols = [
    "household_id", "age", "socioprofessional_class"])

df_persons = df_persons[df_persons["age"] >= 18]

df_persons = df_persons[df_persons["household_id"].isin(
    df_homes["household_id"]
)]

In [ ]:
# Assign age class
df_persons.loc[(df_persons["age"] >= 18) & (df_persons["age"] <= 34), "ac"] = 0 # 18 - 34
df_persons.loc[(df_persons["age"] >= 35) & (df_persons["age"] <= 49), "ac"] = 1 # 35 - 49
df_persons.loc[(df_persons["age"] >= 50) & (df_persons["age"] <= 64), "ac"] = 2 # 50 - 64
df_persons.loc[(df_persons["age"] >= 65), "ac"] = 3 # 65+
df_persons["ac"] = df_persons["ac"].astype(int)

In [ ]:
# Assign socio-professional class
df_persons.loc[df_persons["socioprofessional_class"] == 1, "sc"] = -1 # Agriculture
df_persons.loc[df_persons["socioprofessional_class"] == 2, "sc"] = 0 # CE,Artis,Com
df_persons.loc[df_persons["socioprofessional_class"] == 3, "sc"] = 1 # Cadre
df_persons.loc[df_persons["socioprofessional_class"] == 4, "sc"] = 2 # Prof Int
df_persons.loc[df_persons["socioprofessional_class"] == 5, "sc"] = 3 # Employe
df_persons.loc[df_persons["socioprofessional_class"] == 6, "sc"] = 4 # Ouvrier
df_persons.loc[df_persons["socioprofessional_class"] == 7, "sc"] = 5 # Retraite
df_persons.loc[df_persons["socioprofessional_class"] == 8, "sc"] = 6 # Sans Act
df_persons["sc"] = df_persons["sc"].astype(int)

In [ ]:
# Assign household size class
df_household_size = df_persons.groupby("household_id").size().reset_index(name = "household_size")
df_persons = pd.merge(df_persons, df_household_size, on = "household_id")

df_persons.loc[df_persons["household_size"] == 1, "hc"] = 0 # 1
df_persons.loc[df_persons["household_size"] == 2, "hc"] = 1 # 2
df_persons.loc[df_persons["household_size"] == 3, "hc"] = 2 # 3
df_persons.loc[df_persons["household_size"] >= 4, "hc"] = 3 # 4+
df_persons["hc"] = df_persons["hc"].astype(int)

In the following cell we digitalize the marginal information reported in 

> Gardrat, M., 2019. Méthodologie d’enquête: le découplage de l’achat et de la récupération des marchandises par les ménages (Resarch report). LAET, Lyon, France.

In [ ]:
# LAD by household size class and socioprofessional class (Figure 29)
marginal_hc_sc = np.array([
    [4, 12, 9, 14, 10, 5, 5],
    [27, 18, 15, 4, 16, 6, 13],
    [24, 26, 22, 6, 12, 10, 22],
    [30, 29, 22, 15, 29, 16, 17],
])

# LAD by age class and socioprofessional class (Figure 30)
marginal_ac_sc = np.array([
    [45, 29, 21, 18, 20, 0, 18],
    [30, 29, 19, 14, 22, 0, 11],
    [14, 17, 10, 5, 16, 12, 9],
    [12, 9, 0, 0, 0, 5, 2],
])

# LAD by socioprofessional class (Table 8)
marginal_sc = np.array([
    23.51, 21.19, 19.08, 15.15, 10.31, 9.77, 6.11
])

# Home delivery by socioprofessional class (Table 8)
probability_home_delivery = np.array([
    53.2, 46.8, 40.7, 49.1, 26.3, 56.5, 70.2
]) * 1e-2

# Total number of orders per year
marginal_total = 14.0

In [ ]:
# Sample reference persons and aggregate counts
np.random.seed(random_seed)

sorter = np.arange(len(df_persons))
np.random.shuffle(sorter)

df_persons = df_persons.iloc[sorter]
df_persons = df_persons.drop_duplicates("household_id")

In [ ]:
df = df_persons.groupby([
    "ac", "hc", "sc"
]).size().reset_index(name = "count")

df["weight"] = 1.0
df.loc[df["sc"] == -1, "weight"] = 0.0

In [ ]:
# Apply weighting procedure
for k in tqdm(range(100)): # Run 100 iterations
    
    # Weighting of household size x socioprofessional class
    for hc in range(4):
        for sc in range(7):
            f = (df["hc"] == hc) & (df["sc"] == sc)

            if np.count_nonzero(f) > 0:
                current_weight = (df[f]["weight"] * df[f]["count"]).sum()
                target_weight = marginal_hc_sc[hc, sc] * df[f]["count"].sum()

                if current_weight > 0:
                    factor = target_weight / current_weight
                    df.loc[f, "weight"] *= factor
                    
    # Weighting of household size x socioprofessional class
    for ac in range(4):
        for sc in range(7):
            f = (df["ac"] == ac) & (df["sc"] == sc)

            if np.count_nonzero(f) > 0:
                current_weight = (df[f]["weight"] * df[f]["count"]).sum()
                target_weight = marginal_ac_sc[ac, sc] * df[f]["count"].sum()

                if current_weight > 0:
                    factor = target_weight / current_weight
                    df.loc[f, "weight"] *= factor
    
    # Weighting of socioprofessional class
    for sc in range(7):
        f = df["sc"] == sc

        if np.count_nonzero(f) > 0:
            current_weight = (df[f]["weight"] * df[f]["count"]).sum()
            target_weight = marginal_sc[sc] * df[f]["count"].sum()

            if current_weight > 0:
                factor = target_weight / current_weight
                df.loc[f, "weight"] *= factor
                
    # Weighting of total       
    current_weight = (df["weight"] * df["count"]).sum()
    factor = marginal_total * df["count"].sum() / current_weight
    df["weight"] *= factor

In [ ]:
(df["count"] * df["weight"]).sum() / df["count"].sum() # Should be around 14

In [ ]:
# Assign home delivery probability
df["home_probability"] = 0.0

for sc in range(7):
    f = df["sc"] == sc
    df.loc[f, "home_probability"] = probability_home_delivery[sc]

In [ ]:
# Write output
df.to_parquet(output_path)